In [1]:
import os
import esm
import torch
import biotite.structure.io as bsio

/fs/cbcb-scratch/cys/miniforge3/envs/esmfold/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
directory = "../../example_2_outputs/seqs/"

In [5]:
def average_pLDDT(seq):
    model = esm.pretrained.esmfold_v1()
    model = model.eval().cuda()
    with torch.no_grad():
        output = model.infer_pdb(seq)
    with open("result.pdb", "w") as f:
        f.write(output)
    struct = bsio.load_structure("result.pdb", extra_fields=["b_factor"])
    return struct.b_factor.mean()

In [11]:
for filename in os.listdir(directory):
    if filename.endswith('.fa'):
        print(filename)
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()
        # Process the designed sequences
        output_lines = []
        for i in range(len(lines)):
            line = lines[i].strip()
            if line.startswith('>T='):
                # Extract the designed sequence
                sequence = lines[i + 1].strip()
                
                # Evaluate the designed sequence
                plddt_score = average_pLDDT(sequence)
        
                # Add the pLDDT score to the header
                header = line + f', pLDDT={plddt_score}'
                output_lines.append(header + '\n')
                output_lines.append(sequence + '\n')
            elif line.startswith('>'):
                output_lines.append(line+ '\n')

        # Write the modified sequences to a new file
        with open(file_path, 'w') as file:
            file.writelines(output_lines)